In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from pathlib import Path

In [2]:
# Initialize workflow parameters
num_top = 50
data_path = Path(f'../data/top_{num_top}_companies_by_sector.csv').resolve()
analysis_start = "2024-01-01"
analysis_end = "2025-05-15"
data = pd.read_csv(data_path)
data

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap (B),Country,IPO Year,Volume,Sector,Industry
0,UFPI,UFP Industries Inc. Common Stock,$115.67,-0.7900,-0.678%,7.024022,United States,1993.0,223820,Basic Materials,Forest Products
1,AMWD,American Woodmark Corporation Common Stock,$77.01,0.4800,0.627%,1.159322,United States,1986.0,168642,Basic Materials,Forest Products
2,IPX,IperionX Limited American Depositary Share,$26.52,0.1200,0.455%,0.682213,United States,NaN,50560,Basic Materials,Other Metals and Minerals
3,EU,enCore Energy Corp. Common Shares,$2.91,0.0600,2.105%,0.541518,United States,NaN,1091464,Basic Materials,Other Metals and Minerals
4,USGO,U.S. GoldMining Inc. Common stock,$12.59,1.7800,16.466%,0.156678,United States,2023.0,225247,Basic Materials,Precious Metals
...,...,...,...,...,...,...,...,...,...,...,...
524,VIVK,Vivakor Inc. Common Stock,$0.845,-0.0248,-2.851%,0.028424,United States,NaN,42549,Utilities,Environmental Services
525,SONM,Sonim Technologies Inc. Common Stock,$2.55,-0.0800,-3.042%,0.012423,United States,2019.0,15396,Utilities,Telecommunications Equipment
526,CLRO,ClearOne Inc. (DE) Common Stock,$0.4926,-0.0072,-1.441%,0.011807,United States,NaN,87213,Utilities,Telecommunications Equipment
527,SUNE,SUNation Energy Inc. Common Stock,$1.225,-0.0950,-7.197%,0.002223,United States,NaN,87057,Utilities,Telecommunications Equipment


In [3]:
ticker = data['Symbol'].to_list()
company_name_ticker = data[['Name', 'Symbol']]
market_ticker = '^GSPC'
ticker.append(market_ticker)
company_name_ticker

,Name,Symbol
0,UFP Industries Inc. Common Stock,UFPI
1,American Woodmark Corporation Common Stock,AMWD
2,IperionX Limited American Depositary Share,IPX
3,enCore Energy Corp. Common Shares,EU
4,U.S. GoldMining Inc. Common stock,USGO
...,...,...
524,Vivakor Inc. Common Stock,VIVK
525,Sonim Technologies Inc. Common Stock,SONM
526,ClearOne Inc. (DE) Common Stock,CLRO
527,SUNation Energy Inc. Common Stock,SUNE


In [4]:
stock_price = yf.download(tickers=ticker, start = analysis_start, end = analysis_end)['Close'].dropna(axis = 1)
stock_price

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  530 of 530 completed

21 Failed downloads:
['HYMCW', 'ZEOWW', 'NEOVW', 'NESRW', 'DHCNL', 'SLDPW', 'AMODW', 'NEHCW', 'MVSTW', 'ANNAW', 'FOXXW', 'DFLIW', 'NXPLW', 'ADNWW', 'RMCOW', 'USGOW', 'ARKOW', 'NIOBW']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-01-01 -> 2025-05-15)')
['ROIC', 'QRTEP']: HTTPError('HTTP Error 404: ')
['INFN']: YFTzMissingError('possibly delisted; no timezone found')


Ticker,AAL,AAON,AAPL,ABAT,ABNB,ACDC,ACTG,ADBE,ADI,ADN,...,YORW,Z,ZBRA,ZD,ZEO,ZG,ZION,ZM,ZS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-02,13.44,73.545906,184.290436,4.405,134.479996,7.980,3.92,580.070007,189.526367,6.600,...,37.200626,57.250000,267.980011,67.309998,11.26,56.220001,41.883701,69.150002,212.369995,4742.830078
2024-01-03,12.95,72.759705,182.910522,4.230,133.419998,7.950,3.89,571.789978,185.003113,6.150,...,36.628456,55.230000,252.520004,65.860001,11.28,53.980000,39.853088,67.169998,210.240005,4704.810059
2024-01-04,13.09,73.386673,180.587524,4.250,133.720001,7.640,3.82,567.049988,182.173599,6.000,...,36.250244,54.160000,252.970001,65.570000,11.27,52.750000,40.460377,66.900002,210.330002,4688.680176
2024-01-05,13.60,72.102852,179.862854,4.150,135.979996,8.000,3.84,564.599976,182.643555,5.910,...,35.901123,53.709999,252.690002,65.160004,11.27,52.430000,41.798298,66.959999,209.809998,4697.240234
2024-01-08,14.58,73.346863,184.210999,4.120,140.080002,7.950,3.89,580.549988,185.022690,5.940,...,36.153263,55.669998,261.089996,66.650002,11.28,54.580002,42.263252,68.389999,218.100006,4763.540039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-08,11.03,99.430000,197.231369,1.410,126.389999,3.975,3.54,383.989990,202.779999,3.340,...,32.689999,67.900002,265.369995,32.380001,1.60,66.750000,46.364128,80.250000,233.149994,5663.939941
2025-05-09,11.11,100.230003,198.270004,1.400,127.040001,4.400,3.77,383.279999,207.509995,2.800,...,32.790001,67.930000,266.709991,30.910000,1.67,66.779999,46.106380,81.309998,233.059998,5659.910156
2025-05-12,11.67,104.629997,210.789993,1.340,134.210007,4.940,3.89,395.940002,223.149994,2.740,...,32.099998,71.209999,300.109985,34.700001,1.57,70.080002,49.030781,82.669998,241.479996,5844.189941


In [5]:
returns = stock_price.pct_change().dropna()
ticker = [col for col in stock_price.columns if col != market_ticker]
stock_returns = returns[ticker]
market_returns = returns[market_ticker]
stock_returns

Ticker,AAL,AAON,AAPL,ABAT,ABNB,ACDC,ACTG,ADBE,ADI,ADN,...,XPON,YORW,Z,ZBRA,ZD,ZEO,ZG,ZION,ZM,ZS
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-03,-0.036458,-0.010690,-0.007488,-0.039728,-0.007882,-0.003759,-0.007653,-0.014274,-0.023866,-0.068182,...,0.015968,-0.015381,-0.035284,-0.057691,-0.021542,0.001776,-0.039844,-0.048482,-0.028633,-0.010030
2024-01-04,0.010811,0.008617,-0.012700,0.004728,0.002249,-0.038994,-0.017995,-0.008290,-0.015294,-0.024390,...,0.023576,-0.010326,-0.019374,0.001782,-0.004403,-0.000886,-0.022786,0.015238,-0.004020,0.000428
2024-01-05,0.038961,-0.017494,-0.004013,-0.023529,0.016901,0.047120,0.005236,-0.004321,0.002580,-0.015000,...,-0.007678,-0.009631,-0.008309,-0.001107,-0.006253,0.000000,-0.006066,0.033067,0.000897,-0.002472
2024-01-08,0.072059,0.017253,0.024175,-0.007229,0.030152,-0.006250,0.013021,0.028250,0.013026,0.005076,...,-0.015474,0.007023,0.036492,0.033242,0.022867,0.000887,0.041007,0.011124,0.021356,0.039512
2024-01-09,-0.013717,-0.013840,-0.002263,-0.007281,-0.003926,-0.015094,0.000000,0.009732,0.004127,0.010101,...,0.021611,-0.010998,-0.004131,-0.017810,-0.018305,0.000000,-0.004214,-0.011226,-0.001170,0.037414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-08,0.044508,0.038867,0.006319,0.052239,0.026559,-0.058057,0.153094,-0.002546,0.014610,-0.005952,...,0.062356,-0.014768,0.000295,0.031444,0.023712,-0.030303,0.004666,0.030403,0.020473,-0.001670
2025-05-09,0.007253,0.008046,0.005266,-0.007092,0.005143,0.106918,0.064972,-0.001849,0.023326,-0.161677,...,0.097826,0.003059,0.000442,0.005050,-0.045398,0.043750,0.000449,-0.005559,0.013209,-0.000386
2025-05-12,0.050405,0.043899,0.063146,-0.042857,0.056439,0.122727,0.031830,0.033031,0.075370,-0.021429,...,-0.014851,-0.021043,0.048285,0.125230,0.122614,-0.059880,0.049416,0.063427,0.016726,0.036128


In [6]:
sector_stocks = data[['Symbol', 'Sector']]
sector_stocks_cleaned = sector_stocks[sector_stocks['Symbol'].apply(lambda x: x in ticker)]
sector_stocks_cleaned_with_name = pd.merge(sector_stocks_cleaned, company_name_ticker, left_on='Symbol', right_on = 'Symbol')
sector_stocks_cleaned_with_name

,Symbol,Sector,Name
0,UFPI,Basic Materials,UFP Industries Inc. Common Stock
1,AMWD,Basic Materials,American Woodmark Corporation Common Stock
2,IPX,Basic Materials,IperionX Limited American Depositary Share
3,EU,Basic Materials,enCore Energy Corp. Common Shares
4,USGO,Basic Materials,U.S. GoldMining Inc. Common stock
...,...,...,...
494,VIVK,Utilities,Vivakor Inc. Common Stock
495,SONM,Utilities,Sonim Technologies Inc. Common Stock
496,CLRO,Utilities,ClearOne Inc. (DE) Common Stock
497,SUNE,Utilities,SUNation Energy Inc. Common Stock


In [7]:
residuals = pd.DataFrame(index=stock_returns.index)
for tick in ticker:
    Y = stock_returns[tick]
    X = sm.add_constant(market_returns)
    model = sm.OLS(Y, X).fit()
    residuals[tick] = model.resid
# calculating the residual return 
resid_corr = residuals.corr()
resid_corr

/var/folders/g5/7q0ynznx5hs03d9_7j1h1qbc0000gn/T/ipykernel_5388/1251507463.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  residuals[tick] = model.resid
/var/folders/g5/7q0ynznx5hs03d9_7j1h1qbc0000gn/T/ipykernel_5388/1251507463.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  residuals[tick] = model.resid
/var/folders/g5/7q0ynznx5hs03d9_7j1h1qbc0000gn/T/ipykernel_5388/1251507463.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfor

,AAL,AAON,AAPL,ABAT,ABNB,ACDC,ACTG,ADBE,ADI,ADN,...,XPON,YORW,Z,ZBRA,ZD,ZEO,ZG,ZION,ZM,ZS
AAL,1.000000,0.045487,0.013939,0.062087,0.166405,0.018780,0.112186,-0.015802,0.123420,-0.038666,...,-0.015179,-0.021215,0.030480,0.231893,0.111321,-0.093232,0.034995,0.206321,0.033978,-0.027073
AAON,0.045487,1.000000,-0.244485,0.006797,-0.100677,0.032723,0.121121,-0.064526,-0.042473,-0.002442,...,0.004189,0.018463,0.014138,0.128257,0.110078,0.001087,0.012347,0.072648,0.034495,0.071909
AAPL,0.013939,-0.244485,1.000000,0.022736,0.032831,0.009754,0.022072,-0.025251,0.022508,-0.025850,...,-0.020584,-0.009894,-0.016632,-0.035816,0.020581,0.079602,-0.017742,-0.110755,0.007773,0.002507
ABAT,0.062087,0.006797,0.022736,1.000000,0.026815,0.017245,-0.000058,-0.005761,0.006238,0.100489,...,-0.055443,-0.002327,0.016923,0.018469,0.041217,-0.000568,0.021040,-0.000368,0.097028,0.008665
ABNB,0.166405,-0.100677,0.032831,0.026815,1.000000,-0.030687,0.032206,0.068934,0.120943,0.024653,...,0.230475,-0.074383,0.092531,0.137449,-0.019965,0.051326,0.104024,0.036202,0.112238,-0.008373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEO,-0.093232,0.001087,0.079602,-0.000568,0.051326,0.000623,-0.008191,0.052408,0.087195,0.053508,...,0.074813,-0.019883,0.054418,0.010211,-0.028127,1.000000,0.043804,0.065110,0.001231,-0.009860
ZG,0.034995,0.012347,-0.017742,0.021040,0.104024,-0.063113,-0.023237,0.071467,-0.058644,0.162917,...,-0.016456,0.050418,0.994947,0.030787,0.131158,0.043804,1.000000,-0.016126,0.086628,0.050909
ZION,0.206321,0.072648,-0.110755,-0.000368,0.036202,0.201502,0.249320,-0.072213,0.061609,0.052685,...,-0.071120,0.188690,-0.010364,0.299802,0.265550,0.065110,-0.016126,1.000000,0.103131,0.085398
ZM,0.033978,0.034495,0.007773,0.097028,0.112238,0.072688,0.072347,0.096368,0.000774,0.082414,...,-0.004736,0.047405,0.082223,0.028048,0.214495,0.001231,0.086628,0.103131,1.000000,0.265072


In [8]:
threshold = 0.03
n_component = 2


tsne = TSNE(n_components=n_component, perplexity=15, init='random', random_state=42)
embedding = tsne.fit_transform(resid_corr.values)

In [18]:
n_cluster = 12
kmeans = KMeans(n_clusters=11, random_state=42)
cluster_labels = kmeans.fit_predict(embedding)

In [19]:
df_tsne = pd.DataFrame({
    'Symbol': ticker,
    'PC1': embedding[:,0],
    'PC2': embedding[:,1],
    'clusters': cluster_labels
})
df_tsne

,Symbol,PC1,PC2,clusters
0,AAL,-21.120728,15.491786,2
1,AAON,36.770336,22.869818,3
2,AAPL,17.274752,-6.224302,0
3,ABAT,24.165955,9.641623,10
4,ABNB,0.836618,9.948722,8
...,...,...,...,...
494,ZEO,14.657510,10.859323,8
495,ZG,15.845045,-20.435959,7
496,ZION,-48.787041,7.092076,5
497,ZM,-11.484532,2.957347,2


In [20]:
merged_tsne = pd.merge(df_tsne, sector_stocks_cleaned_with_name, right_on = 'Symbol', left_on = 'Symbol')
merged_tsne

,Symbol,PC1,PC2,clusters,Sector,Name
0,AAL,-21.120728,15.491786,2,Consumer Discretionary,American Airlines Group Inc. Common Stock
1,AAON,36.770336,22.869818,3,Industrials,AAON Inc. Common Stock
2,AAPL,17.274752,-6.224302,0,Technology,Apple Inc. Common Stock
3,ABAT,24.165955,9.641623,10,Basic Materials,American Battery Technology Company Common Stock
4,ABNB,0.836618,9.948722,8,Finance,Airbnb Inc. Class A Common Stock
...,...,...,...,...,...,...
494,ZEO,14.657510,10.859323,8,Miscellaneous,Zeo Energy Corporation Class A Common Stock
495,ZG,15.845045,-20.435959,7,Consumer Discretionary,Zillow Group Inc. Class A Common Stock
496,ZION,-48.787041,7.092076,5,Finance,Zions Bancorporation N.A. Common Stock
497,ZM,-11.484532,2.957347,2,Technology,Zoom Communications Inc. Class A Common Stock


In [21]:
fig = px.scatter(
    merged_tsne,
    x='PC1',
    y='PC2',
    color='clusters',
    hover_name='Symbol',
    hover_data=['Sector', 'Name'],
    title='Interactive t-SNE Clustering of Stocks',
    labels={'PC1': 'PCA Component 1', 'PC2': 'PCA Component 2', 'Name': 'Symbol', 'Sector': 'Sector', 'Name': 'Name'}
)
fig.show()

In [ ]:
sector_list = merged_tsne.groupby(['clusters', 'Sector']).count()
sector_list

In [23]:
df_tsne[df_tsne['Symbol'] == 'TSLA']

,Symbol,PC1,PC2,clusters
440,TSLA,28.988495,11.430099,10
